In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_style("white")

df = pd.read_csv("C:\\Users\\danie\\Documents\\GitHUb\\Machine_Learning_Daniel_Claesson\\data\\Advertising.csv")


In [15]:
X, y = df["TV"], df['Sales']
beta_1, beta_0 = np.polyfit(X, y, deg=1)
#beta_1, beta_0

#def y_hat(beta0, beta_1): # funktion som gör samma som labda funktionen
#    beta_0 + beta_1*x

y_hat = lambda x: beta_0 + beta_1*X

print(f"Intercept beta_0 hat: {beta_0:.4f}")
print(f"Slope beta_1 hat: {beta_1:.4f}")

spend = np.linspace(0, 350)

fig, ax = plt.figure(figsize=(5,3), dpi=100)

sns.scatterplot(data=df, x=TV, y="Sales")
sns.lineplot(x=spend, y=y_hat(spend), color="red")
sns.set(title=("TV..."))

Intercept beta_0 hat: 7.0326
Slope beta_1 hat: 0.0475


TypeError: cannot unpack non-iterable Figure object

<Figure size 500x300 with 0 Axes>

In [16]:
X, y = df.drop("Sales", axis="columns"), df["Sales"]
X.insert(0, "Intercept", 1)

regression_fit = lambda X, y: np.linalg.pinv(X.T @ X) @ X.T @ y
beta_hat = regression_fit(X, y)
predict = lambda x, beta: np.dot(x, beta)

test_sample = [1, 230.1, 37.8, 69.2]

y_hat = predict(test_sample, beta_hat)

print(
    f"TV: $230.1k, Radio: $37.8k, Newspaper: $69.2k predicts {y_hat:.1f} thousand units"
)

ValueError: shapes (4,) and (5,) not aligned: 4 (dim 0) != 5 (dim 0)